# Problem Set 3
In this homework you'll work with NLTK using the NYT comments data you collected in Tutorial 2. You can download that [here](https://github.com/comp-journalism/UMD-J479V-J779V-Spring2017/blob/master/Data/NYT-Comments-12-15-16.csv?raw=true).

In [1]:
import pandas as pd
import nltk

### 1. What are the 25 most frequent unigrams (i.e. words) used across all the comments?
*Hint*: Follow and adapt the basic steps outlined in the tutorial.

In [2]:
import pandas as pd
#Load the csv in the dataframe
file_df = pd.read_csv('NYT-Comments-12-15-16.csv')
#Convert all the comment bodies to form 1 string
print file_df.userDisplayName[0]
filedf_string=', '.join(file_df.commentBody)
filedf_string = unicode(filedf_string, "utf-8")

paulymath


In [3]:
#Convert the string to lower case
filedf_lower = filedf_string.lower() 

table = {
    ord(u'\u2018') : u"'",
    ord(u'\u2019') : u"'",
    ord(u'\u201C') : u'"',
    ord(u'\u201d') : u'"',
    ord(u'\u2026') : u'', 
    ord(u'\u2014') : u'',
}
#Convert the punctuation marks to standard format
filedf_lower = filedf_lower.translate(table)

In [4]:
import string

def remove_punctuation(text):
    #Grab the list of standard punctuation symbols that are provided in the string library
    punctuations = string.punctuation # includes following characters: !"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~

    #Don't strip out apostrophes
    excluded_punctuations = ["'"]
    for p in punctuations:
        if p not in excluded_punctuations:
            #Replace each punctuation symbol by a space
            text = text.replace(p, " ") 
    return text

#Remove unnecessary punctuation
filedf_lower_wopun = remove_punctuation(filedf_lower)

#Remove multiple spaces
filedf_lower_wopun=" ".join(filedf_lower_wopun.split())


In [5]:
from nltk.tokenize import WhitespaceTokenizer

#Create a tokenizer from NLTK which will create tokens based on the whitespace in between words
tokenizer = WhitespaceTokenizer()

#Use the tokenizer on the text string
file_tokens = tokenizer.tokenize(filedf_lower_wopun)

In [6]:
#Use the stopwords in the NLTK library
from nltk.corpus import stopwords
stopword_list = stopwords.words('english')

#Eliminate stopwords from the string
transcript_tokens = [w for w in file_tokens if w not in stopword_list]


In [7]:
#Display the most common 25 unigrams in the strong
from nltk.probability import FreqDist

# Construct the distribution (it will count the number of occurrances of each unique token)
frequency_distribution = FreqDist(transcript_tokens)
print frequency_distribution.most_common(25)

[(u'br', 12455), (u'trump', 4005), (u'would', 1951), (u'people', 1900), (u'one', 1611), (u'us', 1419), (u'like', 1415), (u'vote', 1214), (u"it's", 1053), (u'election', 1043), (u'president', 1035), (u"don't", 1030), (u'get', 1015), (u'even', 945), (u'time', 926), (u'right', 900), (u'think', 891), (u'many', 876), (u'country', 866), (u'electoral', 840), (u'years', 816), (u'mr', 798), (u'states', 792), (u'state', 789), (u'way', 778)]


In [8]:
#Remove unwanted words from the list
del frequency_distribution["br"]
frequency_distribution.most_common(25)

[(u'trump', 4005),
 (u'would', 1951),
 (u'people', 1900),
 (u'one', 1611),
 (u'us', 1419),
 (u'like', 1415),
 (u'vote', 1214),
 (u"it's", 1053),
 (u'election', 1043),
 (u'president', 1035),
 (u"don't", 1030),
 (u'get', 1015),
 (u'even', 945),
 (u'time', 926),
 (u'right', 900),
 (u'think', 891),
 (u'many', 876),
 (u'country', 866),
 (u'electoral', 840),
 (u'years', 816),
 (u'mr', 798),
 (u'states', 792),
 (u'state', 789),
 (u'way', 778),
 (u'republicans', 778)]

### 2. According to the first name of the user who made each comment, how many women vs. men made comments?
To determine the gender of the name, first split the `userDisplayName` field so you isolate the first name. You may have to drop any rows which do not have a userDisplayName. Then use the [Sex Machine library](https://github.com/ferhatelmas/sexmachine/) to tag the sex of the name. Be sure to read and understand the Sex Machine documentation.

In [9]:
#Install the Sex Machine library
! pip install https://pypi.python.org/packages/dd/01/cc5b32af2b3658079736bd865019aeb8db04f9c5764eac72185c276f3aac/SexMachine-0.1.1.tar.gz
    

  Using cached SexMachine-0.1.1.tar.gz
  Running setup.py bdist_wheel for SexMachine: started
  Running setup.py bdist_wheel for SexMachine: finished with status 'done'
  Stored in directory: C:\Users\kavas\AppData\Local\pip\Cache\wheels\0a\9a\19\ec2dd809aeda18aff90f4cf5d9dd1da4b1187fdc7a5948061c
Successfully built SexMachine


You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [10]:
#Extracting the first name from the userDisplayName
file_df["first_name"]=file_df.userDisplayName.str.extract('([A-z]\w{0,})', expand=True)
file_df=file_df.dropna().reset_index(drop=True)

In [11]:
import sexmachine.detector as gender

#Set value as unknown for names not in the list
d = gender.Detector(unknown_value=u"Unknown")
gender_df=[]

#Identify gender based on first name for individuals
print len(file_df)
for i in range(len(file_df)):
    gender=d.get_gender(file_df.first_name[i])
    gender_df.append(gender)

#Copy the list to the dataframe
file_df["Gender"]=gender_df

#Group by gender and display the frequency
print file_df.groupby("Gender").size()

3427
Gender
Unknown          1902
female            455
male              937
mostly_female      64
mostly_male        69
dtype: int64


### 3. What are the average positivity and negativity scores of all comments?
*Hint*: Follow and adapt the basic steps outlined in the tutorial.

In [12]:
#Extract positive and negative terms from the text file
positive_terms = []
f = open('positive_terms.txt', "r")
positive_terms = f.read().splitlines()
f.close()

negative_terms = []
f = open('negative_terms.txt', "r")
negative_terms = f.read().splitlines()
f.close()

In [13]:
# This function will take a string and lowercase it, remove punctuations and spaces, and then tokentize and stem those tokens
import nltk
porter = nltk.PorterStemmer()

file_df = pd.read_csv('NYT-Comments-12-15-16.csv',encoding='utf-8')
def normalize_review_text(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = " ".join(text.split())
    text_tokens = tokenizer.tokenize(text)
    text_tokens = [porter.stem(w) for w in text_tokens if w not in stopword_list]
    return text_tokens

# Apply the function above to the text column
file_df["commentBody"] = file_df["commentBody"].apply(normalize_review_text)

In [14]:
#Calculate the positivity and negativity score for comments
def calculate_positivity(text):
    num_tokens = len(text)
    if num_tokens==0:
        return 0
    num_positive_tokens = 0
    for t in text:
        if t in positive_terms:
            num_positive_tokens = num_positive_tokens + 1
    # The positivity score is the fraction of tokens that were found in the positive dictionary
    return float(num_positive_tokens) / float(num_tokens)

file_df["positivity_score"] = file_df["commentBody"].apply(calculate_positivity)

def calculate_negativity(text):
    num_tokens = len(text)
    if num_tokens==0:
        return 0
    num_negative_tokens = 0
    for t in text:
        if t in negative_terms:
            num_negative_tokens = num_negative_tokens + 1
    # The positivity score is the fraction of tokens that were found in the positive dictionary
    return float(num_negative_tokens) / float(num_tokens)

file_df["negativity_score"] = file_df["commentBody"].apply(calculate_negativity)

In [15]:
#Print the average value of the positivity and negativity score
pos_score=file_df["positivity_score"].mean()
neg_score=file_df["negativity_score"].mean()

print "Average Positivity Score:",pos_score
print "Average Negativity Score:",neg_score

Average Positivity Score: 0.0552719783149
Average Negativity Score: 0.0342588723472


### 4. What are the average positivity and negativity scores for comments marked as editor's selections compared to those not marked as editor's selections?

In [16]:
#Split the dataset into 2 based on editors selection
mask_bool_editor=(file_df["editorsSelection"] == 1)

mask_bool_noteditor=(file_df["editorsSelection"] == 0)

#Calculate the positivity and negativity score for comments
editor_pos_df=file_df.loc[mask_bool_editor]["commentBody"].apply(calculate_positivity)
editor_neg_df=file_df.loc[mask_bool_editor]["commentBody"].apply(calculate_negativity)
noteditor_pos_df=file_df.loc[mask_bool_noteditor]["commentBody"].apply(calculate_positivity)
noteditor_neg_df=file_df.loc[mask_bool_noteditor]["commentBody"].apply(calculate_negativity)

#Print the average value of the positivity and negativity score
editor_pos = editor_pos_df.mean()
editor_neg = editor_neg_df.mean()
noteditor_pos = noteditor_pos_df.mean()
noteditor_neg = noteditor_neg_df.mean()

print "Average Positivity Score for comments marked as editor's selections:",editor_pos
print "Average Negativity Score for comments marked as editor's selections:",editor_neg
print "Average Positivity Score for comments not marked as editor's selections:",noteditor_pos
print "Average Negativity Score for comments not marked as editor's selections:",noteditor_neg

Average Positivity Score for comments marked as editor's selections: 0.0508764913456
Average Negativity Score for comments marked as editor's selections: 0.0399209228364
Average Positivity Score for comments not marked as editor's selections: 0.0553566125976
Average Negativity Score for comments not marked as editor's selections: 0.0341498506199
